# 행정동 매출 예측 모델링
#### 서울시에 존재하는 각 상권의 다음 분기 매출을 예측하는 AI 회귀 모델

## 학습 준비

#### 모델링을 위한 필요 라이브러리 import

In [ ]:
## AI 모델링을 위한 라이브러리인 pycaret, optuna 설치
# 설치 후 런타임 재시작 필요
!pip install -U --pre pycaret -q
!pip install optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.0 MB/s 

In [ ]:
# 구글 드라이브 mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## 필요 라이브러리 import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *

# 한글 변수명 시각화를 위한 폰트 설정
import warnings
warnings.filterwarnings(action='ignore')
plt.rc('font', family='Malgun Gothic')

#### 데이터 불러오기

In [ ]:
# Path 설정
path='/content/drive/MyDrive/KT AIvle 4기 2조 빅프로젝트/preprocessed data/'

In [ ]:
# 1차 전처리 된 학습용 데이터 불러오기 및 shape 확인
data=pd.read_csv('/content/drive/MyDrive/KT AIvle 4기 2조 빅프로젝트/preprocessed data/dong_service_data_predict_fin.csv')
data.shape

(75217, 86)

In [ ]:
# 모든 컬럼 확인
data.columns

기준_년분기_코드, 행정동_코드, 행정동_코드_명, 서비스_업종_코드, 서비스_업종_코드_명, 관공서_수, 은행_수
종합병원_수, 일반_병원_수, 약국_수, 유치원_수, 초등학교_수, 중학교_수, 고등학교_수, 대학교_수, 백화점_수, 슈퍼마켓_수
극장_수, 숙박_시설_수, 공항_수, 철도_역_수, 버스_터미널_수, 지하철_역_수, 버스_정거장_수, 연령대_10_직장_인구_수
연령대_20_직장_인구_수, 연령대_30_직장_인구_수, 연령대_40_직장_인구_수, 연령대_50_직장_인구_수
연령대_60_이상_직장_인구_수, 점포_수, 개업_율, 개업_점포_수, 폐업_률, 폐업_점포_수, 프랜차이즈_점포_수
아파트_단지_수, 아파트_면적_66_제곱미터_미만_세대_수, 아파트_면적_66_제곱미터_세대_수, 아파트_면적_99_제곱미터_세대_수
아파트_면적_132_제곱미터_세대_수, 아파트_면적_165_제곱미터_세대_수, 아파트_가격_1_억_미만_세대_수
아파트_가격_1_억_세대_수, 아파트_가격_2_억_세대_수, 아파트_가격_3_억_세대_수, 아파트_가격_4_억_세대_수
아파트_가격_5_억_세대_수, 아파트_가격_6_억_이상_세대_수, 아파트_평균_면적, 아파트_평균_시가
월_평균_소득_금액, 식료품_지출_총금액, 의류_신발_지출_총금액, 생활용품_지출_총금액, 의료비_지출_총금액
교통_지출_총금액, 교육_지출_총금액, 유흥_지출_총금액, 여가_문화_지출_총금액, 기타_지출_총금액, 음식_지출_총금액
연령대_10_상주인구_수, 연령대_20_상주인구_수, 연령대_30_상주인구_수, 연령대_40_상주인구_수
연령대_50_상주인구_수, 연령대_60_이상_상주인구_수, 아파트_가구_수, 비_아파트_가구_수, 연령대_10_유동인구_수
연령대_20_유동인구_수, 연령대_30_유동인구_수, 연령대_40_유동인구_수, 연령대_50_유동인구_수
연령대_60_이상_유동인구_수, 시간대_00_06_유동인구_수, 시간대_06_11_유동인구_수, 시간대_11_14_유동인구_수
시간대_14_1

In [ ]:
# 학습 전 NAN 존재 확인
data.isna().sum().sum()

0

#### 2차 전처리: 코드명 컬럼 및 불필요한 코드 제거

In [ ]:
# 한글 코드명 컬럼 찾기
name_col=[]
for col in data.columns:
  if '_명' in col:
    # 컬럼명, 타입 출력
    print(col,data[col].dtype)
    # 코드명 컬럼 list에 추가
    name_col.append(col)

행정동_코드_명 object
서비스_업종_코드_명 object


In [ ]:
# 한글 코드명 컬럼 제거 및 불필요한 코드 (기준_년분기_코드) 제거
data=data.drop(name_col+["기준_년분기_코드"],axis=1)
data.shape

(75217, 83)

#### 2차 전처리: One-hot encoding이 필요한 Categorical 컬럼 정의

In [ ]:
# 원본 데이터에서 Categorical 컬럼 찾기
code_cols=[]
for col in data.columns:
  if '_코드'  in col:
    print(col, data[col].dtype)
    code_cols.append(col)

# 파생변수 내 ategorical 컬럼인 분기 코로나_여부 추가
code_cols += ["분기", "코로나_여부"]

행정동_코드 int64
서비스_업종_코드 object


## Auto ML : pycaret regression

#### 학습 설정 및 모델 학습 및 비교 수행

In [ ]:
# import pycaret
from pycaret.regression import *

In [ ]:
# 데이터 전처리 및 실험 설정을 위한 파이프라인 설정을 수행합니다.
exp = setup(
            data=data,
            target = "점포별_예상_평균_매출_금액",
            normalize = True,
            normalize_method = 'zscore', # default는 zscore
            transformation = True,  # transform normality. transformation_method default‘yeo-johnson’
            fold = 5, # default는 10
            fold_shuffle=True,
            categorical_features = code_cols, # 위에서 정의한 one-hot encoding 필요한 컬럼
            max_encoding_ohe=-1, # 0 보다 작으면 생성 컬럼 최대 개수 제한X
            encoding_method= 'onehot',
            session_id = 123, # random state number 지정
            use_gpu = True, # gpu 사용 옵션
            feature_selection = False, # 특징 자동 선택
            )

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Description,Value
0,Session id,123
1,Target,점포별_예상_평균_매출_금액
2,Target type,Regression
3,Original data shape,"(75217, 83)"
4,Transformed data shape,"(75217, 523)"
5,Transformed train set shape,"(52651, 523)"
6,Transformed test set shape,"(22566, 523)"
7,Ordinal features,1
8,Numeric features,78
9,Categorical features,4


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

In [ ]:
# 사용 모델 종류 확인
models()

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [ ]:
# 모델 학습 후 성능 비교 진행
best_model=compare_models(sort='RMSE',n_select=3) #  RMSE기준 정렬. best 3개 모델저장

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,7669780.4015,345437516325246.8125,18414123.8940,0.9322,0.3386,0.2601,360.7020
rf,Random Forest Regressor,8250737.6672,418411924662389.1875,20338073.8932,0.9177,0.3530,0.2820,442.6720
xgboost,Extreme Gradient Boosting,8230372.5000,425667793099161.6250,20487261.6000,0.9157,0.3772,0.2977,32.8340
lightgbm,Light Gradient Boosting Machine,8491235.0928,433821535705036.1250,20692132.1917,0.9143,0.3782,0.3081,33.7540
gbr,Gradient Boosting Regressor,9042632.3233,473216883799526.3750,21636273.7252,0.9067,0.4007,0.3384,98.1040
knn,K Neighbors Regressor,9527368.6000,513675008304742.3750,22552168.4000,0.8990,0.4124,0.3670,46.2940
dt,Decision Tree Regressor,11066273.2017,686797497560726.2500,26154338.2648,0.8644,0.4684,0.3666,38.1080
llar,Lasso Least Angle Regression,23150855.8640,2287372909223975.0000,47766188.7546,0.5488,1.0039,1.8855,30.7000
lasso,Lasso Regression,23238051.1209,2287808606822076.0000,47770033.2941,0.5488,1.0097,1.8944,64.9320
ridge,Ridge Regression,23238109.9780,2288011414975936.0000,47772123.8142,0.5487,1.0083,1.8943,30.2660


Processing:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
# 모델 비교 후 성능이 가장 높은 상위 3개 모델 확인
best_model

[ExtraTreesRegressor(n_jobs=-1, random_state=123),
 RandomForestRegressor(n_jobs=-1, random_state=123),
 XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=123, ...)]

#### 성능 비교 후 best model 3개 튜닝 수행




In [ ]:
# 성능 비교 및 튜닝 실험 조건 정의
exp_sorted = setup(
            data=data,
            target = "점포별_예상_평균_매출_금액",
            normalize = True,
            normalize_method = 'zscore', # default는 zscore
            transformation = True,  # transform normality. transformation_method default‘yeo-johnson’
            fold = 5, # default는 10
            fold_shuffle=True,
            categorical_features = code_cols, # one-hot encoding 필요한 컬럼
            max_encoding_ohe=-1, # 0 보다 작으면 생성 컬럼 최대 개수 제한X
            encoding_method= 'onehot',
            session_id = 123, # random state number 지정
            use_gpu = True, # gpu 사용 옵션
            feature_selection = False, # 특징 자동 선택 default= false
            )

In [ ]:
# best 3개 모델
model_1=best_model[0]
model_2=best_model[1]
model_3=best_model[2]

In [ ]:
## best 3개 모델 각각 튜닝 진행, tuner: optuna 활용
# best1 모델 튜닝 진행 및 튜닝 된 모델 저장
tuned_model_1=tune_model(model_1, search_library='optuna', n_iter=10, optimize='MAE')
save_model(tuned_model_1, '/content/drive/MyDrive/KT AIvle 4기 2조 빅프로젝트/dong_service_pred_model_tune_1')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,14430089.0000,1813463772954624.0000,42584784.0000,0.8077,0.4602,0.3684
1,14070910.0000,1219205521211392.0000,34917124.0000,0.8921,0.4624,0.3796
2,14745887.0000,1329207149854720.0000,36458292.0000,0.9247,0.4446,0.3549
3,14467196.0000,2904055558963200.0000,53889288.0000,0.8465,0.4432,0.3568
4,13602249.0000,1186248223883264.0000,34441956.0000,0.9327,0.4662,0.3399
Mean,14263266.2000,1690436045373440.0000,40458288.8000,0.8807,0.4553,0.3599
Std,394026.5709,647264710411182.2500,7318667.4121,0.0474,0.0095,0.0134


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2024-01-03 01:07:38,519] Searching the best hyperparameters using 24525 samples...
[I 2024-01-03 01:46:31,228] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['관공서_수', '은행_수', '종합병원_수',
                                              '일반_병원_수', '약국_수', '유치원_수',
                                              '초등학교_수', '중학교_수', '고등학교_수',
                                              '대학교_수', '백화점_수', '슈퍼마켓_수', '극장_수',
                                              '숙박_시설_수', '공항_수', '철도_역_수',
                                              '버스_터미널_수', '지하철_역_수', '버스_정거장_수',
                                              '연령대_10_직장_인구_수', '연령대_20_직장_인구_수',
                                              '연령대_30_직장_인구_수', '연령대_40_직장_인구_수',
                                              '연령대_50_직장_인구_수',
                                              '연령대_6...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_

In [ ]:
## best 3개 모델 각각 튜닝 진행, tuner: optuna 활용
# best2 모델 튜닝 진행 및 튜닝 된 모델 저장
tuned_model_2=tune_model(model_2, search_algorithm='optuna', n_iter=10, optimize='MAE')
save_model(tuned_model_2, '/content/drive/MyDrive/KT AIvle 4기 2조 빅프로젝트/dong_service_pred_model_tune_2')

In [1]:
## best 3개 모델 각각 튜닝 진행, tuner: optuna 활용
# best3 모델 튜닝 진행 및 튜닝 된 모델 저장
tuned_model_3=tune_model(model_3, search_algorithm='optuna', n_iter=10, optimize='MAE')
save_model(tuned_model_3, '/content/drive/MyDrive/KT AIvle 4기 2조 빅프로젝트/dong_service_pred_model_tune_3')

In [ ]:
# 튜닝된 모델 결과 확인 - best1
evaluate_model(tuned_model_1)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
# 튜닝된 모델 결과 확인 - best2
evaluate_model(tuned_model_2)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
# 튜닝된 모델 결과 확인 - best3
evaluate_model(tuned_model_3)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
# 튜닝 전후 큰 차이 없거나,
# 오히려 성능이 낮아서 튜닝 전 best모델인 ExtraTreesRegressor 모델 재학습 및 저장
model_1 = create_model('et')
save_model(model_1, '/content/drive/MyDrive/KT AIvle 4기 2조 빅프로젝트/dong_service_pred_model1')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,7545317.3696,278772095841132.8125,16696469.5622,0.9460,0.3365,0.2608
1,7565995.2521,277088789647112.4062,16645984.1898,0.9358,0.3447,0.2601
2,7570497.4790,331389913726273.3750,18204118.0431,0.9338,0.3329,0.2564
3,8005221.0977,544336826956265.2500,23331027.1303,0.8999,0.3405,0.2620
4,7661870.8093,295599955455450.1875,17193020.5448,0.9453,0.3386,0.2613
Mean,7669780.4015,345437516325246.8125,18414123.8940,0.9322,0.3386,0.2601
Std,172454.3560,101348784016947.9375,2521419.7469,0.0169,0.0040,0.0020


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['관공서_수', '은행_수', '종합병원_수',
                                              '일반_병원_수', '약국_수', '유치원_수',
                                              '초등학교_수', '중학교_수', '고등학교_수',
                                              '대학교_수', '백화점_수', '슈퍼마켓_수', '극장_수',
                                              '숙박_시설_수', '공항_수', '철도_역_수',
                                              '버스_터미널_수', '지하철_역_수', '버스_정거장_수',
                                              '연령대_10_직장_인구_수', '연령대_20_직장_인구_수',
                                              '연령대_30_직장_인구_수', '연령대_40_직장_인구_수',
                                              '연령대_50_직장_인구_수',
                                              '연령대_6...
                                                                     '서비스_업종_코드',
                                                                     '분기'],
                   